In [3]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time
import pandas as pd
import string
from rapidfuzz import process
import numpy as np
import os

In [4]:
history = pd.read_csv('lacentrale.csv')
history.columns=['url', '1', '2', 'title', '3', 'price','4','5', '6', '7', 'bike_year', 'mileage']
history = history[['url','title','price','bike_year', 'mileage']]
history.drop_duplicates(subset=['url'], inplace=True)

history.price = history.price.str.replace('€','').str.replace('\s','')

def price_cleaning(price):
    try:
        return int(price)
    except:
        return np.nan

history.price = history.price.apply(price_cleaning)
history.dropna(subset=['price'], inplace=True)
history= history[history.price > 400]

history.mileage = history.mileage.str.replace('km','').str.replace('\s','')

def mileage_cleaning(mileage):
    try:
        return int(mileage)
    except:
        return np.nan

history.mileage = history.mileage.apply(mileage_cleaning)
history.dropna(subset=['mileage'], inplace=True)

def year_cleaning(year):
    try:
        return int(year)
    except:
        return np.nan

history.bike_year = history.bike_year.apply(year_cleaning)
history.dropna(subset=['bike_year'], inplace=True)

moto_database = pd.read_csv('../master_vehicule_list/bikez.csv')
moto_database['brand_model_db'] = moto_database.brand_db + " " + moto_database.model_db

def remove_punctuations(text):
    '''
    remove punctuation in a string
    '''
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def fuzzy_match_title(new_data, moto_database):

    # CLEAN TITLE
    new_data = new_data.dropna(subset=['title'])
    # lower and remove spaces
    new_data.title = new_data.title.str.lower()
    # remove punctuation
    new_data.title = new_data.title.apply(remove_punctuations)
    
    # clean title 
    to_clean = []
    for word in to_clean:
        new_data.title = new_data.title.str.replace(word, '').str.replace('  ', ' ')
        
    new_data.title = new_data.title.str.replace('20\d{2}','', regex=True)
    new_data.title = new_data.title.str.replace('19\d{2}','', regex=True)
        
    def extract_brand(title):
        brand_list = moto_database['brand_db'].unique().tolist()
        for brand in brand_list:
            if brand in title:
                return brand
        
    new_data['brand'] = new_data.title.apply(extract_brand)
    new_data = new_data.dropna(subset=['brand'])
    
    
    def clean_title(title, brand):
        return title.replace(brand, '').replace('  ',' ')
        
    new_data.title = new_data.apply(lambda x: clean_title(x['title'], x['brand']), axis=1)

    # MATCH TITLE
    def choices(year, brand, type_name):
        choices = moto_database[(moto_database.brand_db==brand) & (moto_database.year_db == year)][type_name].unique().tolist()
        return [str(x) for  x in choices]
    
    # unpack results
    def unpack_tuple_name(result):
        try:
            return result[0]
        except:
            return np.nan

    def unpack_tuple_score(result):
        try:
            return result[1]
        except:
            return np.nan
        
    def match_title(choices, to_match):
        return process.extractOne(to_match, choices)

    new_data['fuzzy_match']= new_data.apply(lambda x: match_title(choices(x['bike_year'], x['brand'], 'model_submodel_db'), x['title']), axis=1)
    
    new_data['fuzzy_score'] = new_data['fuzzy_match'].apply(unpack_tuple_score)
    new_data['fuzzy_brand_model'] = new_data['fuzzy_match'].apply(unpack_tuple_name)
    new_data.drop(columns=['fuzzy_match'], inplace=True)
    
    return new_data

startTime = time.time()

history_matched = fuzzy_match_title(history, moto_database)

endTime = time.time()
print(endTime-startTime)

history_matched = history_matched[history_matched.fuzzy_score>75]

history_matched = history_matched.merge(moto_database, how='inner', left_on=['brand', 'bike_year', 'fuzzy_brand_model'], right_on=['brand_db', 'year_db', 'model_submodel_db'])

history_matched.drop(columns=['fuzzy_score','brand_db', 'model_db',
       'model_inv_db', 'model_submodel_db', 'model_submodel_inv_db', 'year_db',
       'category_db', 'engine_type_db', 'power_db',
       'torque_db', 'compression_db', 'cooling_system_db', 'dry_weight_db',
       'power/weight_ratio_db', 'model_size_db', 'model_size_inv_db',
       'brand_model_db'], inplace= True)
history_matched

history_matched.rename(columns={'fuzzy_brand_model':'model' ,'engine_size_db':'engine_size'}, inplace=True)

history_matched.to_csv('lacentrale.csv', index=False)

ValueError: Length mismatch: Expected axis has 8 elements, new values have 12 elements

In [11]:
history = pd.read_csv('lacentrale.csv')
history.iloc[1000].url

'https://www.lacentrale.fr/moto-occasion-annonce-66101618358.html'